## Intro

yeah yeah I'll type this later pls


In [114]:
import requests
import json

js_request = "https://search-a.akamaihd.net/typeahead/suggest/?solrformat=true&rows=20&callback=noCB&q=*:*+AND+schoolid_s:1606&defType=edismax&qf=teacherfirstname_t^2000+teacherlastname_t^2000+teacherfullname_t^2000+autosuggest&bf=pow(total_number_of_ratings_i,2.1)&sort=total_number_of_ratings_i+desc&siteName=rmp&rows=20&start=0&fl=pk_id+teacherfirstname_t+teacherlastname_t+total_number_of_ratings_i+averageratingscore_rf+schoolid_s&fq=&prefix=schoolname_t:\"University+of+Utah\""       

js_response = requests.get(js_request).text
print(js_response)


noCB({
  "responseHeader":{
    "status":0,
    "QTime":14},
  "response":{"numFound":2583,"start":0,"docs":[
      {
        "averageratingscore_rf":3.49,
        "pk_id":261261,
        "total_number_of_ratings_i":173,
        "schoolid_s":"1606",
        "teacherfirstname_t":"David",
        "teacherlastname_t":"Temme"},
      {
        "averageratingscore_rf":4.2,
        "pk_id":335313,
        "total_number_of_ratings_i":169,
        "schoolid_s":"1606",
        "teacherfirstname_t":"Alan",
        "teacherlastname_t":"Sandomir"},
      {
        "averageratingscore_rf":3.8,
        "pk_id":566286,
        "total_number_of_ratings_i":150,
        "schoolid_s":"1606",
        "teacherfirstname_t":"Renee",
        "teacherlastname_t":"Dawson"},
      {
        "averageratingscore_rf":4.3,
        "pk_id":261621,
        "total_number_of_ratings_i":134,
        "schoolid_s":"1606",
        "teacherfirstname_t":"Matthew",
        "teacherlastname_t":"Linton"},
      {
        "averag

After some digging around on the site, I discovered that the list items containing the profesors are actually generated by javascript, and are not in the html source code.  

The "pk_id" value is used to get the individual page of each professor.  So it will be usefull to grab that value and request the actual pages of the profesors to get more of the data I'm looking for.

Also it seems that the site only loads 20 profesors at a time, making the user press a 'load more' button to get the next batch of 20.

In [121]:
import re
import json
import time
import math

clean = re.sub('noCB\(', '', js_response)
clean = re.sub('\);', '', clean)

json_response = json.loads(clean)['response']

total_prof = json_response['numFound']
num_req_needed = math.ceil(total_prof/20)

pk_ids = list()

for i in range(0,num_req_needed):
    next_start = str(i*20)
    js_request = "https://search-a.akamaihd.net/typeahead/suggest/?solrformat=true&rows=20&callback=noCB&q=*:*+AND+schoolid_s:1606&defType=edismax&qf=teacherfirstname_t^2000+teacherlastname_t^2000+teacherfullname_t^2000+autosuggest&bf=pow(total_number_of_ratings_i,2.1)&sort=total_number_of_ratings_i+desc&siteName=rmp&rows=20&start=" + next_start + "&fl=pk_id+teacherfirstname_t+teacherlastname_t+total_number_of_ratings_i+averageratingscore_rf+schoolid_s&fq=&prefix=schoolname_t:\"University+of+Utah\""    
    js_response = requests.get(js_request).text
    
    clean = re.sub('noCB\(', '', js_response)
    clean = re.sub('\);', '', clean)
    
    json_response = json.loads(clean)['response']['docs']
    
    for item in json_response:
        pk_ids.append(item['pk_id'])

    #I'm a nice guy    
    time.sleep(1)

print(len(pk_ids))

2583


In [127]:
print(pk_ids[0:10])

[261261, 335313, 566286, 261621, 261242, 261467, 470792, 673272, 221339, 651891]


Got them all!  

That process took about two minutes.  It may not be feasable to look at every profesor at a University.  In that case I'll just do some random sampling.  

First things first, grab some data for one professor.

In [295]:
from bs4 import BeautifulSoup
import re

url = "https://www.ratemyprofessors.com/ShowRatings.jsp?tid=" + str(pk_ids[0])

response = requests.get(url).content
soup = BeautifulSoup(response, 'html.parser')

rating_breakdown = soup(class_="rating-breakdown")

overall_score = soup.find(class_="breakdown-container quality").div.div.text.strip()
take_again = soup.find(class_=re.compile("breakdown-section takeAgain")).div.text.strip()
difficulty = soup.find(class_="breakdown-section difficulty").div.text.strip()
hot_string = soup.find('figure').img['src']
department = soup.find(class_='result-title').text.strip().split(' ')[3]

if 'hot' in hot_string:
    hot = 1
else:
    hot = 0

most_common_tag = ""
max_tag_count = 0
tags = soup.find(class_="tag-box")

for child in tags.findChildren():
    split_child = child.text.split(" ")
    tag_count_str = split_child[-1]
    tag_count = re.sub('\(','', tag_count_str)
    tag_count = int(re.sub('\)','', tag_count))
    
    if tag_count > max_tag_count:
        max_tag_count = tag_count
        most_common_tag = re.sub('[^A-Za-z ]+', '', child.text).strip()
        


print("Department: " + department + " Overall Score: " + overall_score + " Take Again Chance: " + take_again + " Difficulty: " + difficulty + " Hot: " + str(hot) + " Most Common Tag: " + most_common_tag)

Department: Biology Overall Score: 3.5 Take Again Chance: 65% Difficulty: 3.8 Hot: 0 Most Common Tag: Tough Grader


There is also an ability for each commenter to specify what grade they got in the class.  Which would be a very intersting statistic, however, similar to getting the ids, its loaded from JavaScript requests.  20 a time again.  So it may not be doable for all of the profesors.  

Either way I'll give it a shot